In [1]:
import sys
import os
import random
import gc
import time
import torch
import numpy as np
import pandas as pd
import ast
from tqdm import tqdm
from scipy.sparse import csr_matrix
import itertools
from scipy.stats import spearmanr, pearsonr, kendalltau, rankdata
from sklearn.metrics import ndcg_score
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
import nltk
nltk.download('punkt')
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from SHapRAG import *

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df=pd.read_json("../data/musique/musique_ans_v1.0_train.jsonl", lines=True)

In [3]:
def get_titles(lst):
    # Titles where is_supporting is True
    supporting = [d['paragraph_text'] for d in lst if d.get('is_supporting') == True]
    # Titles where is_supporting is False or missing AND not already in supporting
    others = [d['paragraph_text'] for d in lst if d.get('is_supporting') != True and d['paragraph_text'] not in supporting]
    # Combine: all supporting + as many others as needed to reach 10
    result = supporting + others
    return result[:10]

df.paragraphs=df.paragraphs.apply(get_titles)

In [4]:
df['Sentences'] = df['paragraphs'].apply(
    lambda para_list: [sent for para in para_list for sent in nltk.sent_tokenize(para)]
)

In [ ]:
df.Sentences[24]

In [ ]:
df_save=df_save.drop(columns=['id', 'question_decomposition','paragraphs', 'answer_aliases', 'answerable' ])

In [ ]:
df_save.to_csv()'../data/musique/sen_labeled.csv'

In [ ]:
df_save=pd.read_csv('../data/musique/sen_labeled.csv')

In [ ]:
# df["paragraphs"] = df["paragraphs"].apply(lambda p: p[:5]+ [p[1]] + p[5:])

In [5]:
SEED = 42
# Initialize Accelerator
accelerator_main = Accelerator(mixed_precision="fp16")

# Load Model
if accelerator_main.is_main_process:
    print("Main Script: Loading model...")
# model_path = "mistralai/Mistral-7B-Instruct-v0.3"
model_path = "meta-llama/Llama-3.1-8B-Instruct"
# model_path = "Qwen/Qwen2.5-3B-Instruct"

model_cpu = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_cpu.config.pad_token_id = tokenizer.pad_token_id
    if hasattr(model_cpu, 'generation_config') and model_cpu.generation_config is not None:
        model_cpu.generation_config.pad_token_id = tokenizer.pad_token_id

if accelerator_main.is_main_process:
    print("Main Script: Preparing model with Accelerator...")
prepared_model = accelerator_main.prepare(model_cpu)
unwrapped_prepared_model = accelerator_main.unwrap_model(prepared_model)
unwrapped_prepared_model.eval()
if accelerator_main.is_main_process:
    print("Main Script: Model prepared and set to eval.")

# Define utility cache

accelerator_main.wait_for_everyone()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Main Script: Loading model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.58it/s]


Main Script: Preparing model with Accelerator...
Main Script: Model prepared and set to eval.


In [8]:
# num_questions_to_run=len(df.question)
num_questions_to_run=50
k_values = [1,2,3,4,5]
all_results=[]
LDSs=[]
r2_fm=[]
r2_cc=[]
for i in tqdm(range(num_questions_to_run), disable=not accelerator_main.is_main_process):
    query = df.question[i]
    if accelerator_main.is_main_process:
        print(f"\n--- Question {i+1}/{num_questions_to_run}: {query[:60]}... ---")

    docs=df.Sentences[i]
    utility_cache_base_dir = f"../Experiment_data/musique/{model_path.split('/')[1]}/sentence"
    utility_cache_filename = f"utilities_q_idx{i}.pkl" # More robust naming
    current_utility_path = os.path.join(utility_cache_base_dir, utility_cache_filename)
    
    if accelerator_main.is_main_process: # Only main process creates directories
        os.makedirs(os.path.dirname(current_utility_path), exist_ok=True)
    
    # Initialize Harness
    harness = ContextAttribution(
        items=docs,
        query=query,
        prepared_model=prepared_model,
        prepared_tokenizer=tokenizer,
        accelerator=accelerator_main,
        utility_cache_path=current_utility_path
    )

    print(f'Response: {harness.target_response}')
    print(f'GT: {df.answer[i]}')
    # Compute metrics
    results_for_query = {}
    if accelerator_main.is_main_process:
        m_samples_map = {"L": 364} 
        # m_samples_map = {"L": 128, "XL":256, "XXL":512} 
        T_iterations_map = {"L":40, "XL":50, "XXL":60} 

        for size_key, num_s in m_samples_map.items():
            if 2**len(docs) < num_s and size_key != "L":
                actual_samples = max(1, 2**len(docs)-1 if 2**len(docs)>0 else 1)
            else:
                actual_samples = num_s

            if actual_samples > 0:
                results_for_query[f"ContextCite{actual_samples}"], model_cc = harness.compute_contextcite(num_samples=actual_samples, seed=SEED)
                attributions, ints=harness.compute_spex(sample_budget=actual_samples,max_order=2)
                results_for_query[f"FBII{actual_samples}"]=attributions['fbii']
                results_for_query[f"Spex{actual_samples}"]=attributions['fourier']
                results_for_query[f"FSII{actual_samples}"]=attributions['fsii']
                results_for_query[f"FM_WeightsD{actual_samples}"], F, modelfm = harness.compute_wss(num_samples=actual_samples, seed=SEED, sampling="kernelshap",sur_type="fm", utility_mode="divergence_utility")
                results_for_query[f"FM_Weights{actual_samples}"], F, modelfm = harness.compute_wss(num_samples=actual_samples, seed=SEED, sampling="kernelshap",sur_type="fm")
                # results_for_query[f"BetaShap{actual_samples}"] = harness.compute_beta_shap(num_iterations_max=T_iterations_map[size_key], beta_a=16, beta_b=1, max_unique_lookups=actual_samples, seed=SEED)
                # results_for_query[f"TMC{actual_samples}"] = harness.compute_tmc_shap(num_iterations_max=T_iterations_map[size_key], performance_tolerance=0.001, max_unique_lookups=actual_samples, seed=SEED)

        results_for_query["LOO"] = harness.compute_loo()
        results_for_query["ARC-JSD"] = harness.compute_arc_jsd()

        prob_topk = harness.evaluate_topk_performance(
                                                results_for_query, 
                                                k_values, 
                                                utility_type="probability"
                                            )

        div_topk = harness.evaluate_topk_performance(
                                            results_for_query, 
                                            k_values, 
                                            utility_type="divergence"
                                        )
        
        r2_fm.append([harness.r2_mse(30, modelfm, method='fm')])
        r2_cc.append([harness.r2_mse(30, model_cc, method='cc')])

        LDS = {}
        for i in results_for_query:
            if "FM" in i:
                calculate_LDS = {i:harness.lds(results_for_query[i], 30, utl=True, model=modelfm)}
                LDS.update(calculate_LDS)
            else:
                calculate_LDS = {i:harness.lds(results_for_query[i], 30)}
                LDS.update(calculate_LDS)
        LDS = [{i:harness.lds(results_for_query[i], 30)} for i in results_for_query]

        results_for_query["topk_probability"] = prob_topk
        results_for_query["topk_divergence"] = div_topk
        results_for_query["LDS"] = LDS
        harness.save_utility_cache(current_utility_path)
        
        all_results.append(results_for_query)

  0%|          | 0/50 [00:00<?, ?it/s]


--- Question 1/50: When was the institute that owned The Collegian founded?... ---
Main Process: Attempting to load utility cache from ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx0.pkl...
Successfully loaded 992 cached utility entries.
Response: Houston Baptist University was founded in 1960.
GT: 1960


Computing utilities for ContextCite: 100%|██████████| 364/364 [00:00<00:00, 230797.68it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  2%|▏         | 1/50 [00:18<15:22, 18.82s/it]

Main Process: Saving 992 utility entries to ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx0.pkl...
Save complete.

--- Question 2/50: What year saw the creation of the region where the county of... ---
Main Process: Attempting to load utility cache from ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx1.pkl...
Successfully loaded 1026 cached utility entries.
Response: 1994.
GT: 1994


Computing utilities for ContextCite: 100%|██████████| 364/364 [00:00<00:00, 302861.86it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  4%|▍         | 2/50 [00:45<18:57, 23.69s/it]

...

Main Process: Saving 1020 utility entries to ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx47.pkl...
Save complete.

--- Question 49/50: Who is the spouse of one of the artist who have had concerts... ---
Main Process: Attempting to load utility cache from ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx48.pkl...
Successfully loaded 1044 cached utility entries.
Response: David Furnish is the spouse of Elton John.
GT: David Furnish


Computing utilities for ContextCite: 100%|██████████| 364/364 [00:00<00:00, 242222.22it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
 98%|█████████▊| 49/50 [24:54<00:29, 29.43s/it]

Main Process: Saving 1044 utility entries to ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx48.pkl...
Save complete.

--- Question 50/50: What type of community is the municipality where Norbert Pfr... ---
Main Process: Attempting to load utility cache from ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx49.pkl...
Successfully loaded 1010 cached utility entries.
Response: Lana an der Etsch is a "comune" (municipality).
GT: comune


Computing utilities for ContextCite: 100%|██████████| 364/364 [00:00<00:00, 302262.26it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 50/50 [25:19<00:00, 30.39s/it]

Main Process: Saving 1010 utility entries to ../Experiment_data/musique/Llama-3.1-8B-Instruct/sentence/utilities_q_idx49.pkl...
Save complete.


In [ ]:
precs=np.zeros((len(all_results), len(methods)))
for j, i in enumerate(all_results):
    if j not in [3,6,7,10,13,14,15,16,17,19,20]:
        for n, m in enumerate(methods):
            precs[j][n]=precision(ast.literal_eval(df_save.labels[j]), i[m])

precs.mean(axis=0)

In [7]:
methods = [f'ContextCite{actual_samples}', f'FM_Weights{actual_samples}',f'FM_WeightsD{actual_samples}',f'Spex{actual_samples}',f'FBII{actual_samples}',f'FSII{actual_samples}', 'LOO', 'ARC-JSD']

# Initialize lists
topk_probs = {method: [] for method in methods}
topk_divs = {method: [] for method in methods}
LDSs = {method: [] for method in methods}

# Collect values
for ind, entry in enumerate(all_results):
    for method in methods:
        topk_probs[method].append(entry['topk_probability'][method][2])
        topk_divs[method].append(entry['topk_divergence'][method][2])
        for d in entry['LDS']:
            if method in d:
                LDSs[method].append(d[method])
                break
        

# Compute means
mean_topk_probs = {method: np.mean(topk_probs[method]) for method in methods}
mean_topk_divs = {method: np.mean(topk_divs[method]) for method in methods}
mean_LDSs = {method: np.mean(LDSs[method]) for method in methods}

print("Mean topk_probability:", mean_topk_probs)
print("Mean topk_divergence:", mean_topk_divs)
print("Mean LDS:", mean_LDSs)

Mean topk_probability: {'ContextCite364': 12.571211284399032, 'FM_Weights364': 12.364087232351302, 'FM_WeightsD364': 12.71926569879055, 'Spex364': 13.031725627183913, 'FBII364': 12.251579568088054, 'FSII364': 12.251579568088054, 'LOO': 12.105467292070388, 'ARC-JSD': 11.805027627348899}
Mean topk_divergence: {'ContextCite364': 1.3486342845000832, 'FM_Weights364': 1.361668637958353, 'FM_WeightsD364': 1.4080414159643897, 'Spex364': 1.4066534375193316, 'FBII364': 1.295260504515863, 'FSII364': 1.295260504515863, 'LOO': 1.244968577786883, 'ARC-JSD': 1.2697375269033706}
Mean LDS: {'ContextCite364': 0.8208676307007786, 'FM_Weights364': 0.716360400444939, 'FM_WeightsD364': 0.7853615127919911, 'Spex364': 0.7577306932599748, 'FBII364': 0.7252641083609697, 'FSII364': 0.7252641083609697, 'LOO': 0.640400444938821, 'ARC-JSD': 0.7898998887652948}


In [ ]:
harness.precision([0,1], all)

In [ ]:
import matplotlib.pyplot as plt
def plot_metric(metric_dict, title, ylabel):
    methods = list(metric_dict.keys())
    values = np.array([metric_dict[m] for m in methods])
    
    # Identify the best method (max value assumed to be better)
    best_index = np.argmax(values)
    
    colors = ['gray'] * len(methods)
    colors[best_index] = 'gold'  # Highlight best method

    plt.figure(figsize=(10, 5))
    bars = plt.bar(methods, values, color=colors)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(ylabel)
    plt.title(title)
    
    # Annotate values on top of bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{values[i]:.4f}', ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_metric(mean_topk_probs, "Mean Top-k Probability", "topk_probability")
plot_metric(mean_topk_divs, "Mean Top-k Divergence", "topk_divergence")
plot_metric(mean_LDSs, "Mean LDS", "LDS")

In [ ]:
all_results[0]['FM_WeightsD364']

In [ ]:
[(i,np.array(all_results[1][i]).argsort()) for i in methods]

In [ ]:
df.paragraphs[1]

In [ ]:
all_results[1]

In [ ]:
df.Sentences[10][1]

In [ ]:
[all_results[0][i].argsort() for i in methods]

In [ ]:
all_results[4]['LOO']

In [ ]:
sum(r2_cc)

In [ ]:
for i in range(len(r2_cc)):
    print(r2_cc[i], r2_fm[i])